In [1]:
# 코드 7-3 라이브러리 호출
import torch
import torchtext
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import time

In [2]:
# 코드 7-4 데이터 전처리
start = time.time()
TEXT = torchtext.legacy.data.Field(lower=True, fix_length=200, batch_first=False)
LABEL = torchtext.legacy.data.Field(sequential=False)

In [3]:
# 코드 7-5 데이터셋 준비
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [01:55<00:00, 730kB/s] 


In [4]:
# 코드 7-6 훈련 데이터셋 내용 확인
print(vars(train_data.examples[0]))  # 데이터셋의 내용을 보고자 할 때는 examples를 사용

{'text': ['<br', '/><br', '/>human', 'body', '---', 'wow.<br', '/><br', '/>there', 'are', 'about', '27,000', 'sunrises', 'in', 'human', 'life....<br', '/><br', '/>hardly', 'one', 'thousand', 'sunrises', 'will', 'be', 'watched', 'by', '90%', 'of', 'humans', 'on', 'this', 'planet....<br', '/><br', '/>our', 'days', 'are', 'limited...<br', '/><br', '/>excellent', 'movie', 'for', 'all', 'women....', 'makers', 'of', 'human', 'body...<br', '/><br', '/>thanks', 'and', 'regards.<br', '/><br', '/>'], 'label': 'pos'}


In [5]:
# 코드 7-7 데이터셋 전처리 적용
import string

for example in train_data.examples:
    text = [x.lower() for x in vars(example)["text"]]  # 소문자로 변경
    text = [x.replace("<br", "") for x in text]  # “<br”을 “ ”(공백)으로 변경
    text = [
        "".join(c for c in s if c not in string.punctuation) for s in text
    ]  # 구두점 제거
    text = [s for s in text if s]  # 공백 제거
    vars(example)["text"] = text

In [6]:
# 코드 7-8 훈련과 검증 데이터셋 분리
import random

train_data, valid_data = train_data.split(random_state=random.seed(0), split_ratio=0.8)

In [7]:
# 코드 7-9 데이터셋 개수 확인
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 20000
Number of validation examples: 5000
Number of testing examples: 25000


In [8]:
# 코드 7-10 단어 집합 만들기
TEXT.build_vocab(train_data, max_size=10000, min_freq=10, vectors=None)
LABEL.build_vocab(train_data)

print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 10002
Unique tokens in LABEL vocabulary: 3


In [9]:
# 코드 7-11 테스트 데이터셋의 단어 집합 확인
print(LABEL.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x7f11fc5d40a0>>, {'<unk>': 0, 'pos': 1, 'neg': 2})


In [10]:
# 코드 7-12 데이터셋 메모리로 가져오기
BATCH_SIZE = 64
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

embeding_dim = 100  # 각 단어를 100차원으로 조정(임베딩 계층을 통과한 후 각 벡터의 크기)
hidden_size = 300

train_iterator, valid_iterator, test_iterator = (
    torchtext.legacy.data.BucketIterator.splits(
        (train_data, valid_data, test_data), batch_size=BATCH_SIZE, device=device
    )
)

In [11]:
# 코드 7-13 워드 임베딩 및 RNN 셀 정의
class RNNCell_Encoder(nn.Module):
    def __init__(self, input_dim, hidden_size):
        super(RNNCell_Encoder, self).__init__()
        self.rnn = nn.RNNCell(input_dim, hidden_size)

    def forward(
        self, inputs
    ):  # inputs는 입력 시퀀스로 (시퀀스 길이, 배치, 임베딩(seq,batch, embedding))의 형태를 갖습니다.
        bz = inputs.shape[1]  # 배치를 가져옵니다.
        ht = torch.zeros((bz, hidden_size)).to(
            device
        )  # 배치와 은닉층 뉴런의 크기를 0으로 초기화
        for word in inputs:
            ht = self.rnn(word, ht)
        return ht


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.em = nn.Embedding(len(TEXT.vocab.stoi), embeding_dim)
        self.rnn = RNNCell_Encoder(embeding_dim, hidden_size)
        self.fc1 = nn.Linear(hidden_size, 256)
        self.fc2 = nn.Linear(256, 3)

    def forward(self, x):
        x = self.em(x)
        x = self.rnn(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [12]:
# 코드 7-14 옵티마이저와 손실 함수 정의
model = Net()  # model이라는 이름으로 모델을 객체화
model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [13]:
# 코드 7-15 모델 학습을 위한 함수 정의
def training(epoch, model, trainloader, validloader):
    correct = 0
    total = 0
    running_loss = 0

    model.train()
    for b in trainloader:
        x, y = b.text, b.label  # trainloader에서 text와 label을 꺼내 옵니다.
        x, y = x.to(device), y.to(
            device
        )  # 꺼내 온 데이터가 CPU를 사용할 수 있도록 장치 지정, 반드시 모델과 같은 장치를 사용하도록 지정해야 합니다.
        y_pred = model(x)
        loss = loss_fn(y_pred, y)  # CrossEntropyLoss 손실 함수를 이용하여 오차 계산
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            y_pred = torch.argmax(y_pred, dim=1)
            correct += (y_pred == y).sum().item()
            total += y.size(0)
            running_loss += loss.item()

    epoch_loss = running_loss / len(
        trainloader.dataset
    )  # 누적된 오차를 전체 데이터셋으로 나누어서 에포크 단계마다 오차를 구합니다.
    epoch_acc = correct / total

    valid_correct = 0
    valid_total = 0
    valid_running_loss = 0

    model.eval()
    with torch.no_grad():
        for b in validloader:
            x, y = b.text, b.label
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            y_pred = torch.argmax(y_pred, dim=1)
            valid_correct += (y_pred == y).sum().item()
            valid_total += y.size(0)
            valid_running_loss += loss.item()

    epoch_valid_loss = valid_running_loss / len(validloader.dataset)
    epoch_valid_acc = valid_correct / valid_total

    print(
        "epoch: ",
        epoch,
        "loss： ",
        round(epoch_loss, 3),
        "accuracy:",
        round(epoch_acc, 3),
        "valid_loss： ",
        round(epoch_valid_loss, 3),
        "valid_accuracy:",
        round(epoch_valid_acc, 3),
    )  # 훈련이 진행될 때 에포크마다 정확도와 오차(loss)를 출력
    return epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc

In [14]:
# 코드 7-16 모델 학습
epochs = 5
train_loss = []
train_acc = []
valid_loss = []
valid_acc = []

for epoch in range(epochs):
    epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc = training(
        epoch, model, train_iterator, valid_iterator
    )
    train_loss.append(epoch_loss)  # 훈련 데이터셋을 모델에 적용했을 때의 오차
    train_acc.append(epoch_acc)  # 훈련 데이터셋을 모델에 적용했을 때의 정확도
    valid_loss.append(epoch_valid_loss)  # 검증 데이터셋을 모델에 적용했을 때의 오차
    valid_acc.append(epoch_valid_acc)  # 검증 데이터셋을 모델에 적용했을 때의 정확도

end = time.time()
print(end - start)

epoch:  0 loss：  0.011 accuracy: 0.495 valid_loss：  0.011 valid_accuracy: 0.504
epoch:  1 loss：  0.011 accuracy: 0.503 valid_loss：  0.011 valid_accuracy: 0.492
epoch:  2 loss：  0.011 accuracy: 0.509 valid_loss：  0.011 valid_accuracy: 0.493
epoch:  3 loss：  0.011 accuracy: 0.518 valid_loss：  0.011 valid_accuracy: 0.491
epoch:  4 loss：  0.011 accuracy: 0.527 valid_loss：  0.011 valid_accuracy: 0.508
308.1577982902527


In [15]:
# 코드 7-17 모델 예측 함수 정의
def evaluate(epoch, model, testloader):
    test_correct = 0
    test_total = 0
    test_running_loss = 0

    model.eval()
    with torch.no_grad():
        for b in testloader:
            x, y = b.text, b.label
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            y_pred = torch.argmax(y_pred, dim=1)
            test_correct += (y_pred == y).sum().item()
            test_total += y.size(0)
            test_running_loss += loss.item()

    epoch_test_loss = test_running_loss / len(testloader.dataset)
    epoch_test_acc = test_correct / test_total

    print(
        "epoch: ",
        epoch,
        "test_loss: ",
        round(epoch_test_loss, 3),
        "test_accuracy:",
        round(epoch_test_acc, 3),
    )
    return epoch_test_loss, epoch_test_acc

In [16]:
# 코드 7-18 모델 예측 결과 확인
epochs = 5
test_loss = []
test_acc = []

for epoch in range(epochs):
    epoch_test_loss, epoch_test_acc = evaluate(epoch, model, test_iterator)
    test_loss.append(epoch_test_loss)
    test_acc.append(epoch_test_acc)

end = time.time()
print(end - start)

epoch:  0 test_loss:  0.011 test_accuracy: 0.505
epoch:  1 test_loss:  0.011 test_accuracy: 0.505
epoch:  2 test_loss:  0.011 test_accuracy: 0.505
epoch:  3 test_loss:  0.011 test_accuracy: 0.505
epoch:  4 test_loss:  0.011 test_accuracy: 0.505
361.987530708313
